# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [143]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.tree import export_graphviz
import graphviz

from scipy import stats

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [144]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (l_s / (l_s + r_s)).reshape(l_s.shape[0]) * (
                1 - ((l_c / l_s) ** 2).sum(axis=1)) + (
                       r_s / (l_s + r_s)).reshape(r_s.shape[0]) * (
                       1 - ((r_c / r_s) ** 2).sum(axis=1))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -((l_s / (l_s + r_s)).reshape(l_s.shape[0]) * np.nansum(
            ((l_c / l_s) * np.log2((l_c / l_s))), axis=1) +
                 (r_s / (l_s + r_s)).reshape(r_s.shape[0]) * np.nansum(
                    ((r_c / r_s) * np.log2((r_c / r_s))), axis=1))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (l_s / (l_s + r_s)).reshape(l_s.shape[0]) * (
                1 - (l_c / l_s).max(axis=1)) + (r_s / (l_s + r_s)).reshape(
            r_s.shape[0]) * (
                       1 - (r_c / r_s).max(axis=1))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:max(1, int(np.sqrt(n_feature)))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:max(1, int(np.log2(n_feature)))])

    def __get_feature_ids_N(self, n_feature):
        return np.array(range(n_feature))

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортируем признак x по возрастанию и получаем соответствующий ему y,
        # плюс получаем количество классов.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]

        # Что делает этот блок кода?
        # Получаем правые границы(где значение вектора y меняет класс),
        # предварительо обрезав границы по бокам
        splitted_sorted_y = sorted_y[
                            self.min_samples_split:-self.min_samples_split]
        r_border_ids = \
            np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (
                    self.min_samples_split + 1)
        if len(r_border_ids) != 0:

            # Что делает этот блок кода?
            # Получили число элементов принадлежащих одному класса
            # до правой границы - eq_el_count
            # Создаем матрицу, где число строк - число смен классов,
            # а число столбцов - число классов заполненную нулями -one_hot_code
            # В каждой строке матрицы помечаем,
            # какой класс был до смены, -  единицей
            # Далее вместо единицы в предыдущем классе,
            # получаем число элементов данного класса,
            # предшествущих границе из r_border_ids и записываем
            # это в новую таблицу class_increments
            # Делаем корректировку и в 0 строчке данной таблице дописываем
            # элементры, встречавшиеся в первых min_samples_split
            eq_el_count = r_border_ids - np.append([self.min_samples_split],
                                                   r_border_ids[:-1])
            one_hot_code = np.zeros(
                (r_border_ids.shape[0], np.max(sorted_y) + 1))
            one_hot_code[
                np.arange(r_border_ids.shape[0]), sorted_y[
                    r_border_ids - 1]] = 1
            class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
            class_increments[0] = class_increments[0] + np.bincount(
                sorted_y[:self.min_samples_split],
                minlength=np.max(sorted_y) + 1)
        else:
            splitted_sorted_y = sorted_y
            r_border_ids = \
                np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[
                    0] + 1
            eq_el_count = r_border_ids - np.append([0], r_border_ids[:-1])
            one_hot_code = np.zeros(
                (r_border_ids.shape[0], np.max(sorted_y) + 1))
            one_hot_code[
                np.arange(r_border_ids.shape[0]), sorted_y[
                    r_border_ids - 1]] = 1
            class_increments = one_hot_code * eq_el_count.reshape(-1, 1)                

        # Что делает этот блок кода?
        # Получаем суммарное число элементов каждого
        # типа левее границы - l_class_count -
        # Получаем суммарное число элементов каждого типа правее
        # границы(включая элемент на границе) - r_class_count
        # И соответственно число элементов в каждой строке
        # для  l_class_count - l_sizes, аналогично для r_class_count
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Считаем улучшение меру неопределенности
        # для каждого - получаем вектор gs
        # Записываем в idx индекс наилучшего разбиения по признаку
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # Получает индекс первого элемента в новом классе
        # при наилучшем разбиении
        # Возвращаем наименьшую меру неоприделенности и , средние значения
        # признака, при которых целевой класс "меняется"
        left_el_id = l_sizes[idx][0]
        treshold = (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0
        return gs[idx], treshold

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        y_bincount = np.bincount(y, minlength=np.unique(y).shape[0])
        y_count = np.max(y_bincount / np.sum(y_bincount))
        if (self.max_depth is None):
            is_leaf = (y.shape[0] < self.min_samples_split or
                       y_count >= self.sufficient_share)
        else:
            is_leaf = (depth >= self.max_depth or
                       y.shape[0] < self.min_samples_split or
                       y_count >= self.sufficient_share)
        if (is_leaf):
            mode, count = stats.mode(y)
            self.tree[node_id] = [self.__class__.LEAF_TYPE, mode[0], y_count]
        else:
            feature_ids = self.get_feature_ids(x.shape[1])
            gs_features = np.zeros((feature_ids.shape[0],), dtype=np.float32)
            threshold_features = np.zeros((feature_ids.shape[0],),
                                          dtype=np.float32)
            for i, f_id in enumerate(feature_ids):
                gs_features[i], threshold_features[i] = self.__find_threshold(
                    x[:, f_id], y)
            gs_arg = np.argmin(gs_features)
            gs_min = np.where(gs_features == gs_features[gs_arg])
            cur_idn = np.random.choice(gs_min[0])
            x_left, x_right, y_left, y_right = self.__div_samples(
                x, y, feature_ids[cur_idn], threshold_features[cur_idn])
            if (x_left.shape[0] != 0 and x_right.shape[0] != 0):
                p = y_bincount / np.sum(y_bincount)
                ig = ((1 - (p ** 2).sum()) - gs_features[gs_arg]) * np.sum(
                    y_bincount) / self._X_size
                self.feature_importances_[feature_ids[cur_idn]] += ig
                self.tree[node_id] = [self.__class__.NON_LEAF_TYPE,
                                      feature_ids[cur_idn],
                                      threshold_features[cur_idn]]
                self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1,
                                pred_f)
                self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1,
                                pred_f)
            else:
                if (x_left.shape[0] == 0):
                    mode, count = stats.mode(y_right)
                    y_bincount = np.bincount(y,
                                             minlength=np.unique(y).shape[0])
                    y_count = np.max(y_bincount / np.sum(y_bincount))
                    self.tree[node_id] = [self.__class__.LEAF_TYPE, mode[0],
                                          y_count]
                else:
                    mode, count = stats.mode(y_left)
                    y_bincount = np.bincount(y,
                                             minlength=np.unique(y).shape[0])
                    y_count = np.max(y_bincount / np.sum(y_bincount))
                    self.tree[node_id] = [self.__class__.LEAF_TYPE, mode[0],
                                          y_count]

    def fit(self, x, y):
        self._X_size = x.shape[0]
        self.feature_importances_ = np.zeros((x.shape[1],))
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [145]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [146]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [147]:
%time clf.fit(X_train, y_train)

CPU times: user 6.8 ms, sys: 40 µs, total: 6.84 ms
Wall time: 5.14 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [148]:
%time my_clf.fit(X_train, y_train)

CPU times: user 30.2 ms, sys: 3.99 ms, total: 34.2 ms
Wall time: 31.4 ms


## Проверка качества работы на wine

In [149]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

In [150]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [151]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df.drop_duplicates(subset=['iid']).condtn.value_counts()
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'], axis=1)
df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])
df.field_cd.isnull().sum()
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(
    np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art',
              'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater',
              'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
        'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:,
                             ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                              'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
                'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
        'shar2_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:,
                             ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                              'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
                'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
        df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)


## Проверка скорости работы на Speed Dating Data 

In [152]:
%time clf.fit(X_train, y_train)

CPU times: user 84.2 ms, sys: 2 µs, total: 84.2 ms
Wall time: 83.4 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [153]:
%time my_clf.fit(X_train, y_train)

CPU times: user 563 ms, sys: 0 ns, total: 563 ms
Wall time: 564 ms


## Проверка качества работы на Speed Dating Data

In [154]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5829067582453185

In [155]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.4714901662401143

## Задание 3

In [156]:
stand = Pipeline([
    ("clf", DecisionTreeClassifier(min_samples_split=2, max_depth=10))
        
])
stand.fit(X_train, y_train)
f1_score(y_pred=stand.predict(X_test), y_true=y_test, average='macro')

0.5526376588371198

## Задание 4

In [162]:
features_names = df_pair.columns[1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)
clf = DecisionTreeClassifier(min_samples_split=2)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)
clf_arg = np.argsort(clf.feature_importances_)[::-1]
my_clf_arg = np.argsort(my_clf.feature_importances_)[::-1]
ss = {"clf_key": features_names[clf_arg],
      "clf_feature_importances": clf.feature_importances_[clf_arg],
      "myclf_key": features_names[my_clf_arg],
      "myclf_feature_importances": my_clf.feature_importances_[my_clf_arg]}
feature_ss = pd.DataFrame(data=ss)
display(feature_ss.head(10))

,clf_feature_importances,clf_key,myclf_feature_importances,myclf_key
0,0.081795,int_corr,0.005247,income_f
1,0.035617,age,0.004533,amb1_1_f
2,0.028540,sinc2_1,0.003833,date
3,0.027862,fun1_1,0.003304,fun3_1
4,0.026631,amb1_1_f,0.002823,intel3_1
5,0.026369,income_f,0.002819,tuition_f
6,0.026160,attr1_1_f,0.002636,fun4_1_f
7,0.025459,fun4_1_f,0.002426,int_corr
8,0.025099,field_cd_f,0.002289,fun2_1_f
9,0.023646,shar1_1,0.002253,intel1_1


## Задание 5

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)
parameters = {"min_samples_split": np.arange(2, 10),
             "max_depth": np.arange(1,100)}
clf = GridSearchCV(RandomForestClassifier(), parameters)
clf.fit(X_train, y_train)
clf.best_params_

{'max_depth': 28, 'min_samples_split': 9}